In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn.linear_model as lm
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('./data/ames2000_NAfix.csv')
df.head()

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,Utilities,Lot.Config,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,120,RL,34,3901,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2007,New,Partial,204000
1,20,RL,70,8400,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,MnPrv,None,0,4,2010,WD,Normal,126000
2,85,RL,60,7200,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,4,2009,WD,Normal,133900
3,90,RL,64,7018,Pave,None,Reg,Bnk,AllPub,Inside,...,0,None,None,None,0,6,2009,WD,Alloca,118858
4,60,RL,111,16259,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,None,None,0,9,2006,New,Partial,342643


In [4]:
df.shape

(2000, 80)

In [5]:
df.describe()

,MS.SubClass,Lot.Area,Overall.Qual,Overall.Cond,Year.Built,Year.Remod.Add,X1st.Flr.SF,X2nd.Flr.SF,Low.Qual.Fin.SF,Gr.Liv.Area,...,Wood.Deck.SF,Open.Porch.SF,Enclosed.Porch,X3Ssn.Porch,Screen.Porch,Pool.Area,Misc.Val,Mo.Sold,Yr.Sold,SalePrice
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000
mean,56.945000,10296.665000,6.082000,5.574000,1971.079500,1984.013500,1158.715500,334.380500,4.379000,1497.475000,...,93.058500,46.335000,23.936500,2.116000,15.681500,2.934000,42.51300,6.217500,2007.799500,180240.902000
std,42.699281,8828.208491,1.387169,1.105502,29.985426,20.881771,379.701015,427.519973,42.555738,498.561719,...,126.633128,65.685433,66.144388,20.668119,54.270505,41.080828,440.83108,2.726336,1.316118,78461.927874
min,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,407.000000,0.000000,0.000000,407.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,2006.000000,12789.000000
25%,20.000000,7500.000000,5.000000,5.000000,1954.000000,1965.000000,877.750000,0.000000,0.000000,1126.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,4.000000,2007.000000,129000.000000
50%,50.000000,9465.000000,6.000000,5.000000,1972.000000,1993.000000,1088.000000,0.000000,0.000000,1447.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.00000,6.000000,2008.000000,160000.000000
75%,70.000000,11500.000000,7.000000,6.000000,2000.000000,2004.000000,1391.250000,703.250000,0.000000,1734.000000,...,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.00000,8.000000,2009.000000,213492.500000
max,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,4692.000000,2065.000000,697.000000,5642.000000,...,870.000000,742.000000,1012.000000,320.000000,480.000000,800.000000,15500.00000,12.000000,2010.000000,755000.000000


In [8]:
df.dtypes

MS.SubClass        int64
MS.Zoning         object
Lot.Frontage      object
Lot.Area           int64
Street            object
Alley             object
Lot.Shape         object
Land.Contour      object
Utilities         object
Lot.Config        object
Land.Slope        object
Neighborhood      object
Condition.1       object
Condition.2       object
Bldg.Type         object
House.Style       object
Overall.Qual       int64
Overall.Cond       int64
Year.Built         int64
Year.Remod.Add     int64
Roof.Style        object
Roof.Matl         object
Exterior.1st      object
Exterior.2nd      object
Mas.Vnr.Type      object
Mas.Vnr.Area      object
Exter.Qual        object
Exter.Cond        object
Foundation        object
Bsmt.Qual         object
                   ...  
Bedroom.AbvGr      int64
Kitchen.AbvGr      int64
Kitchen.Qual      object
TotRms.AbvGrd      int64
Functional        object
Fireplaces         int64
Fireplace.Qu      object
Garage.Type       object
Garage.Yr.Blt     object
